In [1]:
from src.newsletters.gmail import Gmail
from src.newsletters.config import NEWSLETTER_AND_PARSER
from src.newsletters.parser.tldr import tldr_parser

In [2]:
NEWSLETTER_AND_PARSER.keys()

dict_keys(['TLDR AI <dan@tldrnewsletter.com>'])

In [3]:
gmail = Gmail()

In [4]:
emails = gmail.fetch_emails(sender='TLDR AI <dan@tldrnewsletter.com>')
len(emails)

4

In [5]:
emails[0]

{'sender': 'TLDR AI <dan@tldrnewsletter.com>',
 'subject': 'Eleven Labs Conversational AI 🤖, Amazon Nova 🌐, Claude on AWS 💻',
 'date_utc': 'Wed, 4 Dec 2024 14:15:46 +0000',
 'id': '1939206ea98a7077',
 'text': "Eleven Labs has launched a new conversational AI service that is more\r\nof an end-to-end product for building conversational\r\nagents.\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\u200c\xa0\r\n\r\n\r\n Sign Up [1] |Advertise [2]|View Online [3] \r\n\r\n\t\tTLDR \r\n\r\n\t\tTOGETHER WITH [Nylas] [4]\r\n\r\nTLDR AI 

In [6]:
news_sources = tldr_parser(emails[0])

In [7]:
news_sources

[{'title': 'ELEVEN LABS CONVERSATIONAL AI',
  'url': 'https://elevenlabs.io/conversational-ai?utm_source=tldrai',
  'source_of_the_news': 'TLDR AI <dan@tldrnewsletter.com>',
  'news_summary': 'Eleven Labs has launched a new conversational AI service that is more of an end-to-end product for building conversational agents. It uses a variety of LLMs on the backend while integrating seamlessly to a wide variety of specialized voices.',
  'date_source': 'Wed, 4 Dec 2024 14:15:46 +0000'},
 {'title': 'CLAUDE MODELS NOW FASTER AND SMARTER ON AWS TRAINIUM2',
  'url': 'https://www.anthropic.com/news/trainium2-and-distillation?utm_source=tldrai',
  'source_of_the_news': 'TLDR AI <dan@tldrnewsletter.com>',
  'news_summary': "Claude models are being optimized for AWS' advanced Trainium2 AI chips. This will enable faster and more efficient performance. Claude 3.5 Haiku is now available on AWS Trainium2 and model distillation in Amazon Bedrock.",
  'date_source': 'Wed, 4 Dec 2024 14:15:46 +0000'},
 

In [17]:
links = emails[0]['text'].split("\r\n\r\n")[-1].split('\r\n')
links

['Links:',
 '------',
 '[1] https://tldr.tech/ai?utm_source=tldrai',
 '[2] https://advertise.tldr.tech/?utm_source=tldrai&utm_medium=newsletter&utm_campaign=advertisetopnav',
 '[3] https://a.tldrnewsletter.com/web-version?ep=1&lc=57aff87c-ae02-11ef-9074-4b7e3c69f0c6&p=0960528c-b232-11ef-b011-f332b2138605&pt=campaign&t=1733321746&s=45b08133287cac9b57c47e1df74765afa60ecccdcecfe4fc9e076407ee6463d4',
 '[4] https://www.nylas.com/personalized-demo/?utm_source=tldr&utm_medium=sponsoredemail&utm_campaign=FY23Q3-TLDR-Developer&utm_content=2023-09-21-issue-demo',
 '[5] https://dashboard-v3.nylas.com/register?utm_source=tldr&utm_medium=sponsoredemail&utm_campaign=FY23Q3-TLDR-Developer&utm_content=2023-09-21-issue-DSU',
 '[6] https://www.nylas.com/products/extractai/?utm_source=tldr&utm_medium=sponsoredemail&utm_campaign=FY23Q3-TLDR-Developer&utm_content=2023-09-21-issue-demo',
 '[7] https://elevenlabs.io/conversational-ai?utm_source=tldrai',
 '[8] https://www.anthropic.com/news/trainium2-and-dist

In [19]:
links[10].split('] ')

['[9',
 'https://aws.amazon.com/blogs/aws/introducing-amazon-nova-frontier-intelligence-and-industry-leading-price-performance/?utm_source=tldrai']